In [ ]:
!pip install deepface

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.5 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116932 sha256=e1c9123c483a7fb140743ffcfc0398846015790f1b64476ccff80bc79ede2ab2
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire


In [ ]:
from deepface import DeepFace

Directory  /root /.deepface created
Directory  /root /.deepface/weights created


In [ ]:
openface_model = DeepFace.build_model(model_name='OpenFace')

In [ ]:
openface

In [ ]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense


In [ ]:
# Define the triplet loss function
def triplet_loss(anchor_emb, positive_emb, negative_emb, margin):
    distance_positive = tf.reduce_sum(tf.square(anchor_emb - positive_emb), axis=1)
    distance_negative = tf.reduce_sum(tf.square(anchor_emb - negative_emb), axis=1)
    loss = tf.reduce_mean(tf.maximum(distance_positive - distance_negative + margin, 0))
    return loss

In [ ]:
# Create custom model by adding an output layer to OpenFace model
embedding_dim = 128  # Adjust according to OpenFace model
openface_output = openface_model.layers[-2].output
output_layer = Dense(embedding_dim, activation=None)(openface_output)
model = Model(inputs=openface_model.input, outputs=output_layer)


In [ ]:
output_layer

<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'dense')>

In [ ]:
# Freeze layers to retain pre-trained weights
for layer in model.layers[:-1]:
    layer.trainable = False

In [ ]:
# Compile the model
learning_rate = 0.001  # Adjust as needed
model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate), loss=None)

In [ ]:
# Create data generators or loaders for anchor, positive, and negative images

# Training loop
num_epochs = 10  # Adjust as needed
margin = 0.2  # Adjust as needed
for epoch in range(num_epochs):
    for anchor_batch, positive_batch, negative_batch in data_generator:
        with tf.GradientTape() as tape:
            anchor_emb = model(anchor_batch)
            positive_emb = model(positive_batch)
            negative_emb = model(negative_batch)
            loss = triplet_loss(anchor_emb, positive_emb, negative_emb, margin)

        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        # Optional: Print loss or other metrics during training

# Save the fine-tuned model for future use
model.save("fine_tuned_openface_model.h5")
